# SQL Queries

We all have a list of queries that are useful that we keep. Maybe it would be good to have these for our team in a notebook

Maybe something for deadlocks

# Deadlocks

<a href="command:workbench.action.terminal.sendSequence?%7B%22text%22%3A%22 .%5c%5cdeadlockmaker.ps1 \u000D \u000D %22%7D">Lets create some deadlocks</a>

and then we can read them from the system_health 



In [1]:
DECLARE @system_health XML

-- Copy ring buffer into an XML variable
SELECT
	@system_health = CAST(xet.target_data AS XML)
FROM
	sys.dm_xe_session_targets xet
	INNER JOIN sys.dm_xe_sessions xe
		ON xe.address = xet.event_session_address
WHERE
	xe.name = 'system_health'
	AND xet.target_name = 'ring_buffer'

-- Get the time stamp and deadlock graph
SELECT
	xed.value('@timestamp','datetime'),
	xed.query('.')
FROM
	@system_health.nodes('RingBufferTarget/event[@name="xml_deadlock_report"]') AS XEventData(xed)

(13 rows affected)

Total execution time: 00:00:19.254

(No column name) (No column name) 2019-09-12 14:09:39.197 <event name="xml_deadlock_report" package="sqlserver" timestamp="2019-09-12T14:09:39.197Z"><data name="xml_report"><type name="xml" package="package0" /><value><deadlock><victim-list><victimProcess id="process90fa59468" /></victim-list><process-list><process id="process90fa59468" taskpriority="0" logused="200" waitresource="RID: 2:3:120:0" waittime="2964" ownerId="71063" transactionname="user_transaction" lasttranstarted="2019-09-12T14:09:20.973" XDES="0x913ec0428" lockMode="U" schedulerid="1" kpid="1292" status="suspended" spid="56" sbid="0" ecid="0" priority="0" trancount="2" lastbatchstarted="2019-09-12T14:09:20.610" lastbatchcompleted="2019-09-12T14:09:20.610" lastattention="1900-01-01T00:00:00.610" clientapp="SQLCMD" hostname="DESKTOP-V8G9S2O" hostpid="10812" loginname="sa" isolationlevel="read committed (2)" xactid="71063" currentdb="2" currentdbname="tempdb" lockTimeout="4294967295" clientoption1="536870944" clientoption2="128056"><executionStack><frame procname="adhoc" line="16" stmtstart="16" stmtend="82" sqlhandle="0x0200000086a188362482cdd2ca19e40115cfd71fa325d2fc0000000000000000000000000000000000000000">
unknown </frame><frame procname="adhoc" line="16" stmtstart="836" stmtend="892" sqlhandle="0x020000007a37972cf2c439015af7a2816f5869981150aade0000000000000000000000000000000000000000">
unknown </frame></executionStack><inputbuf>
IF OBJECT_ID('tempdb..table1') IS NULL
BEGIN
CREATE TABLE table1 (column1 int);
INSERT INTO table1 VALUES (1);
END
IF OBJECT_ID('tempdb..table2') IS NULL
BEGIN
CREATE TABLE table2 (column1 int);
INSERT INTO table2 VALUES (1);
END
BEGIN TRAN
UPDATE table1
SET column1 = 0
DECLARE @waitString varchar(50) = 'WAITFOR DELAY ''00:00:'+ RIGHT('0' + CAST(ABS(CHECKSUM(NEWID())) % 10 AS varchar(2)),2) +''''
EXEC(@waitString)
UPDATE table2
SET column1 = 0
ROLLBACK
BEGIN TRAN
UPDATE table2
SET column1 = 0
SET @waitString = 'WAITFOR DELAY ''00:00:'+ RIGHT('0' + CAST(ABS(CHECKSUM(NEWID())) % 10 AS varchar(2)),2) +''''
EXEC(@waitString)
UPDATE table1
SET column1 = 0
ROLLBACK </inputbuf></process><process id="process90e57b848" taskpriority="0" logused="200" waitresource="RID: 2:1:368:0" waittime="3975" ownerId="71074" transactionname="user_transaction" lasttranstarted="2019-09-12T14:09:28.197" XDES="0x913ef8428" lockMode="U" schedulerid="1" kpid="624" status="suspended" spid="58" sbid="0" ecid="0" priority="0" trancount="2" lastbatchstarted="2019-09-12T14:09:20.567" lastbatchcompleted="2019-09-12T14:09:20.563" lastattention="1900-01-01T00:00:00.563" clientapp="SQLCMD" hostname="DESKTOP-V8G9S2O" hostpid="13980" loginname="sa" isolationlevel="read committed (2)" xactid="71074" currentdb="2" currentdbname="tempdb" lockTimeout="4294967295" clientoption1="536870944" clientoption2="128056"><executionStack><frame procname="adhoc" line="24" stmtstart="16" stmtend="82" sqlhandle="0x02000000d42f0219d0cbdad998dc30c178c61c84ce17200b0000000000000000000000000000000000000000">
unknown </frame><frame procname="adhoc" line="24" stmtstart="1258" stmtend="1314" sqlhandle="0x020000007a37972cf2c439015af7a2816f5869981150aade0000000000000000000000000000000000000000">
unknown </frame></executionStack><inputbuf>
IF OBJECT_ID('tempdb..table1') IS NULL
BEGIN
CREATE TABLE table1 (column1 int);
INSERT INTO table1 VALUES (1);
END
IF OBJECT_ID('tempdb..table2') IS NULL
BEGIN
CREATE TABLE table2 (column1 int);
INSERT INTO table2 VALUES (1);
END
BEGIN TRAN
UPDATE table1
SET column1 = 0
DECLARE @waitString varchar(50) = 'WAITFOR DELAY ''00:00:'+ RIGHT('0' + CAST(ABS(CHECKSUM(NEWID())) % 10 AS varchar(2)),2) +''''
EXEC(@waitString)
UPDATE table2
SET column1 = 0
ROLLBACK
BEGIN TRAN
UPDATE table2
SET column1 = 0
SET @waitString = 'WAITFOR DELAY ''00:00:'+ RIGHT('0' + CAST(ABS(CHECKSUM(NEWID())) % 10 AS varchar(2)),2) +''''
EXEC(@waitString)
UPDATE table1
SET column1 = 0
ROLLBACK </inputbuf></process></process-list><resource-list><ri

In [0]:
CREATE EVENT SESSION [Deadlocks] ON SERVER
ADD EVENT sqlserver.xml_deadlock_report
ADD TARGET package0.ring_buffer(SET max_events_limit=(1000), max_memory=(2048))
WITH (MAX_MEMORY = 2048KB,
      EVENT_RETENTION_MODE = ALLOW_SINGLE_EVENT_LOSS,
      MAX_DISPATCH_LATENCY = 30 SECONDS,
      MAX_EVENT_SIZE = 0KB,
      MEMORY_PARTITION_MODE = NONE,
      TRACK_CAUSALITY = OFF,
      STARTUP_STATE = ON)

In [0]:
SELECT
DATEADD(mi, DATEDIFF(mi, GETUTCDATE(), CURRENT_TIMESTAMP), DeadlockEventXML.value('(event/@timestamp)[1]', 'datetime2')) AS [EventTime],
DeadlockEventXML.value('(//process[@id[//victim-list/victimProcess[1]/@id]]/@hostname)[1]', 'nvarchar(max)') AS HostName,
DeadlockEventXML.value('(//process[@id[//victim-list/victimProcess[1]/@id]]/@clientapp)[1]', 'nvarchar(max)') AS ClientApp,
DB_NAME(DeadlockEventXML.value('(//process[@id[//victim-list/victimProcess[1]/@id]]/@currentdb)[1]', 'nvarchar(max)')) AS [DatabaseName],
DeadlockEventXML.value('(//process[@id[//victim-list/victimProcess[1]/@id]]/@transactionname)[1]', 'nvarchar(max)') AS VictimTransactionName,
DeadlockEventXML.value('(//process[@id[//victim-list/victimProcess[1]/@id]]/@isolationlevel)[1]', 'nvarchar(max)') AS IsolationLevel,
DeadlockEventXML.query('(event/data[@name="xml_report"]/value/deadlock)[1]') AS DeadLockGraph,
DeadlockEventXML
FROM
(
SELECT
XEvent.query('.') AS DeadlockEventXML,
Data.TargetData
FROM
(
SELECT
CAST(target_data AS XML) AS TargetData
FROM sys.dm_xe_session_targets st
JOIN sys.dm_xe_sessions s ON s.address = st.event_session_address
WHERE s.name = 'Deadlocks' AND
st.target_name = 'ring_buffer'
) AS Data
CROSS APPLY TargetData.nodes('RingBufferTarget/event[@name="xml_deadlock_report"]') AS XEventData(XEvent)
) AS DeadlockInfo 

ADd a new block